## Connect to Azure Machine Learning Workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ai.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [default azure authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for this tutorial. Check the [configuration notebook](../../jobs/configuration.ipynb) for more details on how to configure credentials and connect to a workspace.
```

In [8]:
!pip install azure-ai-ml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 15.1 MB/s eta 0:00:00 0:00:01
Using cached colorama-0.4.6-py2.py3-none-any.whl (25 kB)


In [10]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# enter details of your AML workspace
subscription_id = "71a4020f-108b-496f-a693-d74af9fa2677"
resource_group = "nuance-team"
workspace = "nuance-experiments"

# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

## Reading/writing data in a job

In [13]:
from azure.ai.ml import command
from azure.ai.ml.entities import Data, JobService
from azure.ai.ml import Input, Output
from azure.ai.ml.constants import AssetTypes

dtype = 'sliced8bit'

inputs = {
    "input_data": Input(type=AssetTypes.URI_FILE, path=f"azureml://subscriptions/71a4020f-108b-496f-a693-d74af9fa2677/resourcegroups/nuance-team/workspaces/nuance-experiments/datastores/workspaceblobstore/paths/UI/2023-11-04_170833_UTC/train/{dtype}/",)
}

job = command(
    code="../src",  # local path where the code is stored
    command="python train.py --input_data ${{inputs.input_data}}",
    inputs=inputs,
    environment="spotDetectionEnv:3",
    compute="josh-gpu",
)

# submit the command
returned_job = ml_client.create_or_update(job)
# get a URL for the status of the job
returned_job.studio_url

'https://ml.azure.com/runs/nifty_ear_51w56g287h?wsid=/subscriptions/71a4020f-108b-496f-a693-d74af9fa2677/resourcegroups/nuance-team/workspaces/nuance-experiments&tid=49a830aa-c92e-47d5-bb7e-264801d69db4'

## With Debugger Attached

In [7]:
from azure.ai.ml import command
from azure.ai.ml.entities import Data, JobService, VsCodeJobService
from azure.ai.ml import Input, Output
from azure.ai.ml.constants import AssetTypes

dtype = 'sliced8bit'

inputs = {
    "input_data": Input(type=AssetTypes.URI_FILE, path=f"azureml://subscriptions/71a4020f-108b-496f-a693-d74af9fa2677/resourcegroups/nuance-team/workspaces/nuance-experiments/datastores/workspaceblobstore/paths/UI/2023-11-04_170833_UTC/train/{dtype}/",)
}

job = command(
    code="../src",  # local path where the code is stored
    command="python -m debugpy --listen localhost:5678 --wait-for-client train.py --input_data ${{inputs.input_data}}" ,
    inputs=inputs,
    environment="datalabeling:8",
    compute="joshs-cpu-big",
    services={
        "my_vscode": VsCodeJobService(port=5678),
    },
    
)

# submit the command
returned_job = ml_client.create_or_update(job)
# get a URL for the status of the job
returned_job.studio_url

Uploading nuance_experiments (15.87 MBs): 100%|██████████| 15868063/15868063 [00:16<00:00, 989912.21it/s] 


Readonly attribute status will be ignored in class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.JobService'>


'https://ml.azure.com/runs/cool_watch_sktydlc6ym?wsid=/subscriptions/71a4020f-108b-496f-a693-d74af9fa2677/resourcegroups/nuance-team/workspaces/nuance-experiments&tid=49a830aa-c92e-47d5-bb7e-264801d69db4'